In [2]:
import pandas as pd

In [3]:
riot = pd.read_csv('riot_total_data_v2.csv')

In [4]:
riot.head()

,championId,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,perk3,perk4,primaryStyle,perk5,perk6,subStyle,gameMode,gameVersion,gameDuration,gameId
0,92,BRONZE,TOP,200,False,0,13,10,16360,10,...,9104,8299,8000,8275,8210,8200,CLASSIC,13.20.536.2230,2074,6753818491
1,21,BRONZE,BOTTOM,100,False,1,5,5,9812,17,...,8345,8347,8300,8233,8236,8200,CLASSIC,13.17.528.2266,1569,6678104058
2,81,BRONZE,BOTTOM,100,False,9,7,8,20404,8,...,9103,8014,8000,8304,8345,8300,CLASSIC,13.24.547.9214,1641,6855651107
3,145,BRONZE,BOTTOM,200,False,3,7,2,6146,3,...,9104,8014,8000,8139,8106,8100,CLASSIC,13.12.514.8523,1172,6544329012
4,23,BRONZE,TOP,100,False,1,5,2,7117,6,...,9104,8299,8000,8444,8242,8400,CLASSIC,13.18.530.4653,1126,6711170124


In [5]:
riot.shape

(6293924, 60)

In [6]:

convert_riot = riot.sort_values(by=['gameId', 'teamId']).reset_index(drop=True)


gameId로 게임을 구별합니다. player10명을 찾고 이외 데이터는 버리도록 했습니다.

In [7]:
overlab = convert_riot['gameId'].value_counts().reset_index()
overlab = overlab.loc[overlab["count"] == 10]['gameId']
overlab.drop_duplicates()

756       6916212397
757       6916298326
758       6917185536
759       6916212513
760       6917195660
             ...    
609904    6722930774
609905    6722364227
609906    6722363616
609907    6722340908
609908    6723239714
Name: gameId, Length: 609153, dtype: int64

In [8]:
convert_riot = convert_riot.loc[convert_riot['gameId'].isin(overlab)]
convert_riot.head(20)

,championId,tier,position,teamId,win,kills,deaths,assists,totalDamageDealtToChampions,visionScore,...,perk3,perk4,primaryStyle,perk5,perk6,subStyle,gameMode,gameVersion,gameDuration,gameId
0,35,SILVER,UTILITY,100,True,8,4,16,21868,57,...,8210,8237,8200,8345,8347,8300,CLASSIC,13.6.499.7758,1813,6432686803
1,145,SILVER,BOTTOM,100,True,8,8,12,19570,29,...,9103,8014,8000,8345,8304,8300,CLASSIC,13.6.499.7758,1813,6432686803
2,127,SILVER,MIDDLE,100,True,16,4,11,29888,32,...,8138,8106,8100,8226,8236,8200,CLASSIC,13.6.499.7758,1813,6432686803
3,54,SILVER,TOP,100,True,4,1,13,20758,38,...,8210,8237,8200,8345,8352,8300,CLASSIC,13.6.499.7758,1813,6432686803
4,79,SILVER,JUNGLE,100,True,7,3,16,24528,24,...,8138,8135,8100,8210,8232,8200,CLASSIC,13.6.499.7758,1813,6432686803
5,22,SILVER,BOTTOM,200,False,7,9,8,19472,18,...,9103,8014,8000,8410,8345,8300,CLASSIC,13.6.499.7758,1813,6432686803
6,56,SILVER,JUNGLE,200,False,2,12,7,14032,22,...,9104,8014,8000,8138,8106,8100,CLASSIC,13.6.499.7758,1813,6432686803
7,8,SILVER,MIDDLE,200,False,3,7,1,19308,20,...,8210,8236,8200,8304,8347,8300,CLASSIC,13.6.499.7758,1813,6432686803
8,711,SILVER,UTILITY,200,False,6,9,10,23617,74,...,8138,8106,8100,8226,8210,8200,CLASSIC,13.6.499.7758,1813,6432686803
9,26,SILVER,TOP,200,False,2,6,4,17328,24,...,8210,8237,8200,8347,8321,8300,CLASSIC,13.6.499.7758,1813,6432686803


In [9]:
# Redefine the function to correctly reshape data for each game into a single row
def reshape_game_data_corrected(group):
    # Flatten the group data by concatenating each player's data into a single row
    reshaped = pd.concat([group.iloc[i] for i in range(len(group))], axis=0)
    # reshaped.index = [f'player_{i + 1}_{col}' for i, col in enumerate(reshaped.index)]
    return reshaped.to_frame().T

In [10]:
# # Group the data by 'gameId' and apply the corrected transformation
# grouped_data_corrected = convert_riot.groupby('gameId').apply(reshape_game_data_corrected)

# # Reset index to clean up the dataframe
# reshaped_data_final_corrected = grouped_data_corrected.reset_index(drop=True)

# # Show the structure of the transformed data
# reshaped_data_final_corrected.info(), reshaped_data_final_corrected.head()
# reshaped_data_final_corrected.to_csv('reshaped_data_final_corrected.csv',mode = 'a', index=False)

In [12]:
# 게임 ID 별로 데이터 그룹화 및 변형하여 파일에 바로 저장
for name, group in convert_riot.groupby('gameId'):
    reshaped = reshape_game_data_corrected(group)
    with open('reshape_total_riot.csv', 'a', newline='') as f:
        reshaped.to_csv(f, header=f.tell()==0, index=False)

KeyboardInterrupt: 